In [0]:
STAGES = [
    "1. Group Stage",
    "2. Round of 16",
    "3. Quarter-finals",
    "4. Semi-finals",
    "5. 3rd Place Final",
    "6. Final"
]

if not spark.catalog.tableExists("pipeline_config"):
    target_stage = STAGES[0]
    print(f" STARTING NEW SIMULATION: {target_stage}")
else:
    last_processed_stage = spark.table("pipeline_config").first()["current_stage"]
    
    try:
        current_index = STAGES.index(last_processed_stage)
        next_index = current_index + 1
        
        if next_index < len(STAGES):
            target_stage = STAGES[next_index]
            print(f" ADVANCING TO: {target_stage}")
        else:
            print(" TOURNAMENT FINISHED! All stages are complete.")
            dbutils.notebook.exit("FINISHED")
            
    except ValueError:

        print(" Unknown stage found in config. Resetting to start.")
        target_stage = STAGES[0]


spark.sql(f"CREATE OR REPLACE TABLE pipeline_config AS SELECT '{target_stage}' as current_stage")

print("=" * 40)
print(f" EXECUTING PIPELINE FOR: {target_stage}")
print("=" * 40)

print("    1. Ingestion (Bronze)...")
dbutils.notebook.run("01_ingestion_check", 600)

print("    2. Cleaning (Silver)...")
dbutils.notebook.run("02_silver_cleaning", 600)

print("    3. KPI Updates (Gold)...")
dbutils.notebook.run("03_gold_kpis", 600)

print(f"\n SUCCESS: {target_stage} is now Live on the Dashboard.")

 ADVANCING TO: 3. Quarter-finals
 EXECUTING PIPELINE FOR: 3. Quarter-finals
    1. Ingestion (Bronze)...
    2. Cleaning (Silver)...
    3. KPI Updates (Gold)...

 SUCCESS: 3. Quarter-finals is now Live on the Dashboard.
